In [1]:
import pandas as pd
import numpy as np
from dateutil import parser

pd.options.mode.chained_assignment = None 

days_to_rate = 21

# Function to handle parsing and errors
def safe_parse(date):
    "Handles parsing dates when some may be NA"
    try:
        return parser.parse(str(date))
    except Exception as e:
        # Handle the exception or return a default value
        return pd.NaT  # pd.NaT represents a missing value for datetime types

**Dictionary that takes a state abbreviation to its full name**

In [2]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

**Combining data from 2022 polls and results**

In [3]:
#Past results, needed for getting error/bias
results_2022_house = (
    pd.read_csv("../../cleaned_data/House Historical.csv")
    .assign(office_type = 'U.S. House', 
            district = lambda x: np.where(x['district'] == 0, 1, x['district']))
    .rename({'state_po': 'state'}, axis = 'columns')
)

#Senate, 2022 Gubernatorial data not ready yet :( will be soon!

results_2022 = (
    results_2022_house
    .query('year == 2022')
    .filter(['state', 'district', 'office_type', 'margin'])
    .assign(state = lambda x: x['state'].map(states))
)

polls_2022 = pd.concat([
    pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/president_polls_historical.csv"), 
    pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/senate_polls_historical.csv"), 
    pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/house_polls_historical.csv"),
    pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/governor_polls_historical.csv")
])

#Conducting necessary cleaning on 2022 polls
polls_2022.columns = polls_2022.columns.str.strip()
polls_2022_filtered = (
    polls_2022
    .assign(
        election_date=lambda x: x['election_date'].apply(safe_parse),
        end_date=lambda x: x['end_date'].apply(safe_parse),
        days_before_election=lambda x: (x['election_date'] - x['end_date']).dt.days
    )
    .query('not @pd.isna(pollster_rating_id) & cycle > 2020 &  (not ranked_choice_reallocated) & days_before_election < @days_to_rate')
    .assign(party=lambda x: np.where(x['party'].isin(['DEM', 'REP']), x['party'], 'IND'),
            partisan=lambda x: np.where(x['partisan'].isna(), 'NO', x['partisan']))
    .groupby(by=['poll_id', 'pollster_rating_id', 'pollster', 'question_id', 'methodology', 'state', 'sample_size', 'office_type', 'population', 'partisan',
             'seat_number', 'days_before_election', 'party'], as_index=False, dropna=False)
    .agg(party_sum=pd.NamedAgg(column ='pct', aggfunc='sum'))
    .pivot_table(index=['poll_id', 'pollster_rating_id', 'pollster', 'question_id', 'methodology', 'state', 'sample_size', 'office_type', 'population', 'partisan',
             'seat_number', 'days_before_election'], columns='party', values='party_sum')
    .reset_index()
    .rename({'seat_number': 'district'}, axis = 'columns')
    .query('not @pd.isna(DEM) & not @pd.isna(REP)')
)

polls_2022_completed = (
    pd.merge(left=results_2022, right=polls_2022_filtered, how='right', on=['state', 'district', 'office_type'])
    .query('not @pd.isna(margin)')
    .assign(pred_margin = lambda x: x['DEM'] - x['REP'],
            bias = lambda x: (x['pred_margin']) - x['margin'],
            error = lambda x: np.abs(x['bias']),
            office_type = lambda x: x['office_type'].str.replace("U.S. ", ""), 
            year = 2022)
    .rename({"margin": "actual_margin"}, axis='columns')
    .filter(['year', 'state', 'district', 'office_type', 'poll_id', 'pollster_rating_id', 'pollster', 'question_id', 'methodology',
            'sample_size', 'population', 'partisan', 'days_before_election', 'pred_margin', 'actual_margin', 'bias', 'error'])

)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

**Combining 2022 Polls with Polls from Earlier Years**

In [ ]:
#Converting between names for office types
office_type_dict = {
    'House-G': 'House', 
    'Sen-G': 'Senate',
    'Gov-G': 'Governor'
}

#Cleaning polls from before 2022
prior_polls = (
    pd.read_csv("../../data/raw_polls.csv")
    .assign(
        electiondate=lambda x: x['electiondate'].apply(safe_parse),
        polldate=lambda x: x['polldate'].apply(safe_parse),
        days_before_election=lambda x: (x['electiondate'] - x['polldate']).dt.days, 
        pred_margin = lambda x: x['cand1_pct'] - x['cand2_pct'],
        actual_margin = lambda x: x['cand1_actual'] - x['cand2_actual'],
        bias = lambda x: x['pred_margin'] - x['actual_margin'], 
        error = lambda x: np.abs(x['bias']), 
        state = lambda x: x['location'].str.split("-", expand=True)[0], 
        district = lambda x: x['location'].str.split("-", expand=True)[1], 
        population = 'lv' #538 says all these polls were LV
    )
    .query('state != "US" & cand1_party == "DEM" & cand2_party == "REP"') # only looking at DvR polls
    .assign(district = lambda x: x['district'].fillna(0), 
            state = lambda x: x['state'].map(states), 
            office_type = lambda x: x['type_simple'].map(office_type_dict), 
            partisan = lambda x: x['partisan'].fillna("NO"))
    .rename({'samplesize': 'sample_size'}, axis='columns')
    .filter(['year', 'state', 'district', 'office_type', 'poll_id', 'pollster_rating_id', 'pollster', 'question_id', 'methodology',
             'sample_size', 'population', 'partisan', 'days_before_election', 'pred_margin', 'actual_margin', 'bias', 'error'])
)

all_past_polls = pd.concat([prior_polls, polls_2022_completed])
all_past_polls['partisan'] = np.where(all_past_polls['partisan'] == "REP", "R", all_past_polls['partisan'])
all_past_polls['partisan'] = np.where(all_past_polls['partisan'] == "DEM", "D", all_past_polls['partisan'])
all_past_polls['partisan'] = np.where((all_past_polls['partisan'] == "IND") | (all_past_polls['partisan'] == "DEM,REP"), 
                                      "NO", all_past_polls['partisan'])

all_past_polls['methodology'].value_counts()

methodology
Live Phone                                             5722
IVR                                                    2599
Online Panel                                           2100
IVR/Online Panel                                        878
IVR/Live Phone                                          280
IVR/Text                                                157
Live Phone/Online Panel                                 143
Text-to-Web/Online Ad                                   139
IVR/Live Phone/Text/Online Panel/Email                  113
Online Panel/Text-to-Web                                107
IVR/Online Panel/Text-to-Web                            102
Live Phone/Text-to-Web                                   91
Probability Panel                                        78
Online Panel/Online Ad                                   75
IVR/Text-to-Web                                          71
Online Ad                                                69
IVR/Online Panel/Text-to-Web

**Saving final poll compilation**

In [ ]:
all_past_polls.to_csv("../../cleaned_data/PollsforRating.csv")

In [ ]:
all_past_polls.head()

,state,district,office_type,poll_id,pollster_rating_id,pollster,question_id,methodology,sample_size,population,partisan,days_before_election,pred_margin,actual_margin,bias,error,year
0,Nebraska,0,Senate,32945,391.0,YouGov,39543,Online Panel,721.0,lv,NO,39.0,-27.0,-32.851535,5.851535,5.851535,NaN
1,Nebraska,0,Senate,33777,391.0,YouGov,40963,Online Panel,681.0,lv,NO,15.0,-29.0,-32.851535,3.851535,3.851535,NaN
2,Michigan,0,Senate,33068,263.0,Public Policy Polling,126637,IVR/Online Panel,687.0,lv,NO,59.0,7.0,13.285311,-6.285311,6.285311,NaN
3,Michigan,0,Senate,33061,323.0,Suffolk University,39666,Live Phone,500.0,lv,NO,57.0,8.8,13.285311,-4.485311,4.485311,NaN
4,Michigan,0,Senate,33037,582.0,Denno Research,39642,Live Phone,600.0,lv,NO,53.0,7.0,13.285311,-6.285311,6.285311,NaN
